<div style='background-color: #87ceeb;
    border: 0.5em solid black;
    border-radius: 0.5em;
    padding: 1em;'>
    <h2>Activité</h2>
    <h1>Voyage en France</h1>
</div>

**(1)** Avant de commencer, installer le module `geodaisy`, disponible via le _Python Package Index_.

In [ ]:
import sys
!{sys.executable} -m pip install geodaisy

On veut définir un module `voyage` dans le fichier `voyage.py` à télécharger [ici](https://ntoulzac.github.io/Cours-NSI-Terminale/modularite/donnees/voyage.py). Le but de ce module est de fournir des fonctions permettant de repérer des lieux sur une carte ou d'y représenter des itinéraires.

Les cartes seront construites grâce au module `folium` et les éléments géographiques seront obtenus en interrogeant l'API *Base Adresse Nationale* du gouvernement français et l'API <a href="https://geoservices.ign.fr/documentation/geoservices/itineraires.html"><i>Itinéraires</i></a> de l'Institut Géographique National.

Deux fonctions sont prédéfinies dans le module :
- `_coord_BAN` prend en entrée un couple `lieu` au format `(adresse, code_postal)`, où `adresse` est une chaîne de caractères et `code_postal` un entier. Elle retourne un couple au format `(latitude, longitude)` contenant les coordonnées géographiques du `lieu` obtenues grâce à l'API *Base Adresse Nationale*.
- `_itineraire_IGN` prend en entrée deux couples `dep` et `arr` au format `(latitude, longitude)`, où `latitude` et `longitude` sont deux flottants. Elle retourne un dictionnaire dont les clés sont `distance`, `duree` et `geojson`. On obtient en effet, grâce à l'API *Itinéraires* de l'IGN, à la fois la longueur, la durée et la représentation spatiale du trajet le plus rapide entre `dep` et `arr`.

**(2)** Votre travail consiste, dans le module `voyage`, à définir les fonctions et procédures suivantes :

*(a)* `creer_carte`. Elle renvoie une carte créée avec `folium`, centrée sur le point de coordonnées (46,513°N ; 2,576°E) avec un niveau de zoom 6.

In [ ]:
def creer_carte():
    return folium.Map(location=[46.513, 2.576], tiles="OpenStreetMap", zoom_start=6)

*(b)* `enregistrer_carte` prend en paramètres d'entrée une `carte` et un `nom_de_fichier`. Elle enregistre la carte dans le fichier `nom_de_fichier.html`.

In [ ]:
def enregistrer_carte(carte, nom_de_fichier):
    carte.save(f"{nom_de_fichier}.html")

*(c)* `placer_marqueur` prend en paramètres d'entrée une `carte` et un `lieu` au format `(adresse, code_postal)`. Elle place sur la `carte` un marqueur au `lieu` désigné et renvoie la carte ainsi modifiée.

In [ ]:
def placer_marqueur(carte, lieu):
    folium.Marker(_coord_BAN(lieu)).add_to(carte)
    return carte

*(d)* `tracer_itineraire` prend en paramètres d'entrée une `carte` et deux lieux `depart` et `arrivee` au format `(adresse, code_postal)`. Elle dessine sur la `carte` l'itinéraire le plus court entre les deux lieux, place un marqueur sur les deux lieux et renvoie la `carte` ainsi modifiée.

In [ ]:
def tracer_itineraire(carte, depart, arrivee):
    carte = placer_marqueur(carte, depart)
    carte = placer_marqueur(carte, arrivee)
    geojson = _itineraire_IGN(_coord_BAN(depart), _coord_BAN(arrivee))["geojson"]
    folium.GeoJson(geojson).add_to(carte)
    return carte

*(e)* `tracer_itineraire_multiple` prend en paramètres d'entrée une `carte` et un tableau `lieux` contenant des lieux (au moins deux) au format `(adresse, code_postal)`. Elle dessine sur la `carte` l'itinéraire le plus court passant (dans l'ordre) par tous les lieux désignés et renvoie la carte ainsi modifiée.

In [ ]:
def tracer_itineraire_multiple(carte, lieux):
    for k in range(len(lieux)-1):
        carte = tracer_itineraire(carte, lieux[k], lieux[k+1])
    return carte

**(3)** Ecrire la documentation du module et des fonctions et procédures.

On peut tester les fonctions définies dans le module `voyage` (module complet à télécharger [ici](https://ntoulzac.github.io/Cours-NSI-Terminale/modularite/notebooks/corr/voyage.py)) en exécutant la cellule suivante.

In [ ]:
import voyage
depart = ("16 avenue Henri Mondor", 15000)
intermediaire = ("24 avenue des Landais", 63170)
arrivee = ("173 boulevard de Strasbourg", 94130)
ma_carte = voyage.creer_carte()
ma_carte = voyage.tracer_itineraire_multiple(ma_carte, [depart, intermediaire, arrivee])
voyage.enregistrer_carte(ma_carte, "exemple")

In [ ]:
ma_carte